# Obtaining Data

- Palmitoylation site information is obtained from SwissPalm
- sites.txt is obtained and convert to sites.csv
- sites.csv is saved at https://github.com/sonluongvu/Palm_structure

Open sites.csv

In [1]:
! wget 'https://github.com/sonluongvu/Palm_structure/raw/main/sites.csv'

--2022-01-25 18:16:40--  https://github.com/sonluongvu/Palm_structure/raw/main/sites.csv
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/sonluongvu/Palm_structure/main/sites.csv [following]
--2022-01-25 18:16:40--  https://raw.githubusercontent.com/sonluongvu/Palm_structure/main/sites.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3260327 (3.1M) [text/plain]
Saving to: ‘sites.csv’

sites.csv           100%[===================>]   3.11M  --.-KB/s    in 0.07s   

2022-01-25 18:16:41 (47.0 MB/s) - ‘sites.csv’ saved [3260327/3260327]



In [2]:
import pandas as pd
import numpy as np
sites_path = '/content/sites.csv'
sites = pd.read_csv(sites_path)
sites.shape

(7452, 26)

Number of palmitoylation sites based on organisms

In [3]:
sites.organism.value_counts()

Mus musculus                                                            4328
Homo sapiens                                                            2691
Rattus norvegicus                                                        160
Saccharomyces cerevisiae (strain ATCC 204508 / S288c)                     46
Bos taurus                                                                38
Arabidopsis thaliana                                                      30
Oryctolagus cuniculus                                                     30
Torpedo californica                                                       13
Drosophila melanogaster                                                   12
Canis familiaris                                                           8
Semliki forest virus                                                       8
Hepatitis E virus genotype 3 (isolate Human/United States/US2)             8
Sindbis virus                                                              8

- Most of the provided palmitoylation sites above are not verified by experiments
- Listed of verified sites ID is obtained from SwissPalm database

Obtain list of verified palm sites for Homo sapiens

In [10]:
! wget 'https://github.com/sonluongvu/Palm_structure/raw/main/Rattus_norvegicus/palmprot_by_organism_targeted_sites_3.txt'

--2022-01-25 18:23:33--  https://github.com/sonluongvu/Palm_structure/raw/main/Rattus_norvegicus/palmprot_by_organism_targeted_sites_3.txt
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/sonluongvu/Palm_structure/main/Rattus_norvegicus/palmprot_by_organism_targeted_sites_3.txt [following]
--2022-01-25 18:23:33--  https://raw.githubusercontent.com/sonluongvu/Palm_structure/main/Rattus_norvegicus/palmprot_by_organism_targeted_sites_3.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1997 (2.0K) [text/plain]
Saving to: ‘palmprot_by_organism_targeted_sites_3.txt’

palmprot_by_organis 100%[========

In [11]:
Rattus_norvegicus_id_path = '/content/palmprot_by_organism_targeted_sites_3.txt'
Rattus_norvegicus_id = pd.read_csv(Rattus_norvegicus_id_path)
Rattus_norvegicus_id.shape

(163, 2)

Obtain the list of id

In [13]:
Rattus_norvegicus_list = []
for i in range(len(Rattus_norvegicus_id.index)):
  x = str(Rattus_norvegicus_id.iloc[i][0])
  y = 'SPALMS'+ x[7:]
  Rattus_norvegicus_list.append(y)

len(Rattus_norvegicus_list)

163

Obtain info for Homo_sapiens_list from sites

In [14]:
Rattus_norvegicus_sites = sites[sites['id'].isin(Rattus_norvegicus_list)]
Rattus_norvegicus_sites.uniprot_ac.value_counts()

P21707    15
P60881    12
Q99M64    11
P31016     8
Q8VGC3     8
P21818     6
P49799     5
P47863     4
P23977     4
P07936     4
Q63803     4
Q5XIL2     4
Q9Z1L1     4
Q03555     4
P70496     4
P63095     3
Q00960     3
Q9Z2S9     3
Q00959     3
P59215     3
Q62747     3
P10824     3
Q9JM47     2
P49187     2
Q9JM59     2
O35095     2
Q8CFN2     2
P20171     2
O70511     2
P30560     2
Q05683     2
P19490     2
P70498     2
P16235     2
P29994     2
Q9JHU6     2
Q9Z1Y0     1
P32301     1
O08589     1
P04775     1
P07174     1
P08290     1
P19491     1
Q920K5     1
P31596     1
P33535     1
P09483     1
Q9WTW1     1
P25023     1
P21555     1
Name: uniprot_ac, dtype: int64

Obtain protein sequence

In [15]:
! pip install biopython

     |████████████████████████████████| 2.3 MB 5.4 MB/s 


In [16]:
import requests as r
from Bio import SeqIO
from io import StringIO 

Uniprot_ID = Rattus_norvegicus_sites['uniprot_ac']
Seq_list = []
for cID in Uniprot_ID:
  baseUrl="http://www.uniprot.org/uniprot/"
  currentUrl=baseUrl+cID+".fasta"
  response = r.post(currentUrl)
  cData=''.join(response.text)

  Seq=StringIO(cData)
  pSeq=list(SeqIO.parse(Seq,'fasta'))
  Seq_list.append(pSeq)

In [17]:
protein_seq_list = []
for record in Seq_list:
  sequence = 200*'-' + str(record[0].seq) +200*'-'
  protein_seq_list.append(sequence)
Rattus_norvegicus_sites['Protein_seq'] = protein_seq_list
Rattus_norvegicus_sites

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,id,hit_id,uniprot_ac,uniprot_id,uniprot_description,organism,main_gene_name,gene_names,study_id,study_organism,study_cell_type,study_subcellular_fraction,pmid,authors,title,year,hit_techniques,large_scale,hit_list,acyl_type,isoform,pos,uncertain_pos,pats,apts,site_techniques,Protein_seq
25,SPALMS53,SPALMH5929,P09483,ACHA4_RAT,Neuronal acetylcholine receptor subunit alpha-4,Rattus norvegicus,Chrna4,"Acra4,Chrna4",31,Homo sapiens,tsA 201,Total cell lysate,22593584,Amici SA et al.,A highly conserved cytoplasmic cysteine residu...,2012,"Cleavage of thioester bonds with NH2OH,Truncat...",No,None,palmitate,NaN,273,No,NaN,NaN,"Point mutation of substrate,Labeling of substr...",----------------------------------------------...
26,SPALMS57,SPALMH5933,Q05683,DCE2_RAT,Glutamate decarboxylase 2 (EC 4.1.1.15) (65 kD...,Rattus norvegicus,Gad2,"Gad65,Gad2",34,Mus musculus,COS7,Total cell lysate,8132714,Shi Y et al.,Amino acid residues 24-31 but not palmitoylati...,1994,"Cleavage of thioester bonds with NH2OH,Labelin...",No,None,palmitate,NaN,30,No,NaN,NaN,NaN,----------------------------------------------...
27,SPALMS58,SPALMH5933,Q05683,DCE2_RAT,Glutamate decarboxylase 2 (EC 4.1.1.15) (65 kD...,Rattus norvegicus,Gad2,"Gad65,Gad2",34,Mus musculus,COS7,Total cell lysate,8132714,Shi Y et al.,Amino acid residues 24-31 but not palmitoylati...,1994,"Cleavage of thioester bonds with NH2OH,Labelin...",No,None,palmitate,NaN,45,No,NaN,NaN,NaN,----------------------------------------------...
28,SPALMS59,SPALMH5934,P63095,GNAS2_RAT,Guanine nucleotide-binding protein G(s) subuni...,Rattus norvegicus,Gnas,"Gnas1,Gnas",35,Rattus norvegicus,COS,Total cell lysate,8347607,Degtyarev MY et al.,The G protein alpha s subunit incorporates [3H...,1993,"Cleavage of thioester bonds with NH2OH,Labelin...",No,None,palmitate,NaN,3,No,NaN,NaN,NaN,----------------------------------------------...
106,SPALMS222,SPALMH9333,Q00960,NMDE2_RAT,"Glutamate receptor ionotropic, NMDA 2B (GluN2B...",Rattus norvegicus,Grin2b,Grin2b,164,Homo sapiens,HEK293T,Total cell lysate,19874789,Hayashi T et al.,Dual palmitoylation of NR2 subunits regulates ...,2009,"Labeling of substrate with [3H]palmitic acid,P...",No,None,palmitate,NaN,849,No,NaN,NaN,"Point mutation of substrate,Labeling of substr...",----------------------------------------------...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7372,SPALMS5731,SPALMH26506,Q62747,SYT7_RAT,Synaptotagmin-7 (Protein Syt7) (Synaptotagmin ...,Rattus norvegicus,Syt7,Syt7,1097,Chlorocebus aethiops,COS,Total cell lysate,15355980,Kang R et al.,Presynaptic trafficking of synaptotagmin I is ...,2004,"Labeling of substrate with [3H]palmitic acid,P...",No,None,palmitate,NaN,35,Yes,NaN,NaN,"Point mutation of substrate,Labeling of substr...",----------------------------------------------...
7373,SPALMS5732,SPALMH26506,Q62747,SYT7_RAT,Synaptotagmin-7 (Protein Syt7) (Synaptotagmin ...,Rattus norvegicus,Syt7,Syt7,1097,Chlorocebus aethiops,COS,Total cell lysate,15355980,Kang R et al.,Presynaptic trafficking of synaptotagmin I is ...,2004,"Labeling of substrate with [3H]palmitic acid,P...",No,None,palmitate,NaN,38,Yes,NaN,NaN,"Point mutation of substrate,Labeling of substr...",----------------------------------------------...
7374,SPALMS5733,SPALMH26506,Q62747,SYT7_RAT,Synaptotagmin-7 (Protein Syt7) (Synaptotagmin ...,Rattus norvegicus,Syt7,Syt7,1097,Chlorocebus aethiops,COS,Total cell lysate,15355980,Kang R et al.,Presynaptic trafficking of synaptotagmin I is ...,2004,"Labeling of substrate with [3H]palmitic acid,P...",No,None,palmitate,NaN,41,Yes,NaN,NaN,"Point mutation of substrate,Labeling of substr...",----------------------------------------------...
7420,SPALMS5637,SPALMH26405,Q9JHU6,STMN3_RAT,Stathmin-3 (SCG10-like protein),Rattus norvegicus,Stmn3,"Sclip,Stmn3",79,Rattus norvegicus,Cortical neuron,Total cell lysate,21471001,Levy AD et al.,Subcellular Golgi localization of stathmin fam...

In [19]:
columns = ['id', 'uniprot_ac', 'pos', 'protein_seq']
Rattus_norvegicus_info = Rattus_norvegicus_sites.filter(columns, axis = 1)
Rattus_norvegicus_info['protein_seq'] = protein_seq_list
Rattus_norvegicus_info.shape

(160, 4)

In [20]:
Rattus_norvegicus_info.to_csv('Rattus_norvegicus_info.csv', index = False)
type(Seq_list[1])

list

Obtain 100 aa upstream and 100 aa downstream